In [1]:
# docs and experiment results can be found at https://docs.cleanrl.dev/rl-algorithms/ppo/#ppo_pettingzoo_ma_ataripy
import random
import sys
import time
from dataclasses import dataclass
from typing import Dict, Tuple, TypeVar

import gymnasium
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tyro
from colorama import Fore, Style, init
from pufferlib.emulation import PettingZooPufferEnv
from pufferlib.vector import Serial
from torch.distributions.categorical import Categorical
from tqdm import tqdm

sys.path.append("../../..")
from lasertag import LasertagAdversarial  # noqa: E402
from syllabus.core import PettingZooTaskWrapper

ActionType = TypeVar("ActionType")
AgentID = TypeVar("AgentID")
AgentType = TypeVar("AgentType")
EnvTask = TypeVar("EnvTask")
AgentTask = TypeVar("AgentTask")
ObsType = TypeVar("ObsType")

IndentationError: unexpected indent (module.py, line 31)

In [ ]:
@dataclass
class Config:
    # Experiment configuration
    exp_name: str = "lasertag_"
    wandb_project_name: str = "syllabus"
    wandb_entity: str = "rpegoud"
    logging_dir: str = "."
    torch_deterministic: bool = True
    cuda: bool = True
    track: bool = False
    capture_video: bool = False

    # Experiment setup
    env_id: str = "pong_v3"
    total_timesteps: int = int(1e5)
    seed: int = 0

    # Algorithm specific arguments
    gamma: float = 0.995
    gae_lambda: float = 0.95
    anneal_lr: bool = True
    norm_adv: bool = True
    target_kl = None  # type: float
    # Adam args
    adam_lr: float = 1e-4
    adam_eps: float = 1e-5
    # PPO args
    clip_coef: float = 0.2
    vf_coef: float = 0.5
    clip_vloss: bool = True
    max_grad_norm: float = 0.5
    ent_coef: float = 0.0
    num_workers: int = 8
    num_minibatches: int = 4
    rollout_length: int = 256
    update_epochs: int = 5

In [ ]:

def parse_args() -> Config:
    init(autoreset=True)  # Initialize colorama
    config = tyro.cli(Config)
    config.batch_size = int(config.num_workers * config.rollout_length)
    print(
        f"{Fore.RED}{Style.BRIGHT} Setting BATCH_SIZE to: NUM_WORKERS * ROLLOUT_LENGTH"
        f"= {config.batch_size}"
    )
    config.minibatch_size = int(config.batch_size // config.num_minibatches)
    print(
        f"{Fore.RED}{Style.BRIGHT} Setting MINIBATCH_SIZE to:"
        f"BATCH_SIZE * NUM_MINIBATCHES= {config.minibatch_size}"
    )
    log_config(config)
    print(
        f"{Fore.BLUE}{Style.BRIGHT} RUNNING PPO ON {config.env_id} "
        f"FOR {config.total_timesteps} TIMESTEPS ..."
    )
    return config


def log_config(config: Config) -> None:
    config_dict = config.__dict__
    print(f"{Fore.GREEN}{Style.BRIGHT} Config:")
    log_str = " \n".join(
        [
            f"{key.replace('_', ' ').capitalize()}: {Fore.GREEN}{value}{Style.RESET_ALL}"
            for key, value in config_dict.items()
        ]
    )
    print(log_str)


def log_rewards(r_batch: torch.tensor, run, step: int) -> None:
    r_logs = {f"agent_{agent_idx}": reward for agent_idx, reward in enumerate(r_batch)}

    run["charts/cumulative_rewards_per_update"].append(r_logs, step=step)


def batchify(x, device=None):
    """Converts PZ style returns to batch of torch arrays."""
    # convert to list of np arrays
    x = np.stack([x[a] for a in x], axis=0)
    # convert to torch
    x = torch.tensor(x)

    if device is not None:
        x = x.to(device)

    return x


def unbatchify(x, possible_agents: np.ndarray):
    """Converts np array to PZ style arguments."""
    x = x.cpu().numpy()
    x = {agent: x[idx] for idx, agent in enumerate(possible_agents)}

    return x


class LasertagParallelWrapper(PettingZooTaskWrapper):
    """
    Wrapper ensuring compatibility with the PettingZoo Parallel API.

    Lasertag Environment:
        * Action shape:  `n_agents` * `Discrete(5)`
        * Observation shape: Dict('image': Box(0, 255, (`n_agents`, 3, 5, 5), uint8))
    """

    def __init__(self, n_agents, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n_agents = n_agents
        self.task = None
        self.episode_return = 0
        self.possible_agents = [f"agent_{i}" for i in range(self.n_agents)]
        self.env.agents = self.possible_agents
        self.n_steps = 0
        self.env.render_mode = "human"

    def observation_space(self, agent):
        env_space = self.env.observation_space["image"]
        # Remove agent dimension
        return gymnasium.spaces.Box(
            low=env_space.low[0],
            high=env_space.high[0],
            shape=env_space.shape[1:],
            dtype=env_space.dtype,
        )

    def action_space(self, agent):
        return gymnasium.spaces.Discrete(5)

    def __getattr__(self, name):
        """
        Delegate attribute lookup to the wrapped environment if the attribute
        is not found in the LasertagParallelWrapper instance.
        """
        return getattr(self.env, name)

    def _np_array_to_pz_dict(self, array: np.ndarray) -> Dict[str, np.ndarray]:
        """
        Returns a dictionary containing individual observations for each agent.
        Assumes that the batch dimension represents individual agents.
        """
        out = {}
        for idx, value in enumerate(array):
            out[self.possible_agents[idx]] = value
        return out

    def _singleton_to_pz_dict(self, value: bool) -> Dict[str, bool]:
        """
        Broadcasts the `done` and `trunc` flags to dictionaries keyed by agent id.
        """
        return {agent: value for agent in self.agents}

    def reset(
        self, seed: int = None
    ) -> Tuple[Dict[AgentID, ObsType], Dict[AgentID, dict]]:
        """
        Resets the environment and returns a dictionary of observations
        keyed by agent ID.
        """
        self.env.seed(seed)
        obs = self.env.reset_random()  # random level generation
        pz_obs = self._np_array_to_pz_dict(obs["image"])
        return pz_obs, {}

    def step(
        self,
        action: Dict[AgentID, ActionType],
    ) -> Tuple[
        Dict[AgentID, ObsType],
        Dict[AgentID, float],
        Dict[AgentID, bool],
        Dict[AgentID, bool],
        Dict[AgentID, dict],
    ]:
        """
        Takes inputs in the PettingZoo (PZ) Parallel API format, performs a step and
        returns outputs in PZ format.
        """
        action = batchify(action)
        obs, rew, done, info = self.env.step(action)
        obs = obs["image"]
        trunc = False  # there is no `truncated` flag in this environment
        self.task_completion = self._task_completion(obs, rew, done, trunc, info)
        # convert outputs back to PZ format
        obs, rew = map(self._np_array_to_pz_dict, [obs, rew])
        done, trunc, info = map(
            self._singleton_to_pz_dict, [done, trunc, self.task_completion]
        )
        # info["agent_id"] = agent_task
        self.n_steps += 1
        return self.observation(obs), rew, done, trunc, info

In [ ]:
def make_env():
    env = LasertagAdversarial()
    env = LasertagParallelWrapper(env=env, n_agents=2)
    env = PettingZooPufferEnv(env)
    return env

In [ ]:
class Agent(nn.Module):
    def __init__(self, envs) -> None:
        super().__init__()

        self.conv = nn.Sequential(
            self.layer_init(
                nn.Conv2d(
                    envs.single_observation_space.shape[0],
                    out_channels=16,
                    kernel_size=3,
                    stride=1,
                )
            ),
            nn.Flatten(),
            nn.ReLU(),
        )
        self.lstm = nn.LSTM(input_size=16 * 3 * 3, hidden_size=256, batch_first=True)
        self.lstm_init()

        self.mlp = nn.Sequential(
            nn.ReLU(),
            self.layer_init(nn.Linear(256, 32)),
            nn.ReLU(),
        )
        self.actor = self.layer_init(
            nn.Linear(32, envs.single_action_space.n), scale=0.01
        )
        self.critic = self.layer_init(nn.Linear(32, 1), scale=1)

    def get_states(self, x, lstm_states, done):
        # x shape: (num_envs, *obs_shape)
        batch_size = x.size(0)
        hidden = self.conv(x / 255.0)
        hidden = hidden.view(batch_size, 1, -1)  # shape: (num_envs, 1, n_features)

        # Reshape LSTM states
        h_states, c_states = lstm_states
        h_states = h_states.view(1, batch_size, -1)
        c_states = c_states.view(1, batch_size, -1)

        # Apply the LSTM
        hidden, (new_h_states, new_c_states) = self.lstm(hidden, (h_states, c_states))

        # Reset LSTM state if done
        done = done.view(batch_size, 1, 1)
        new_h_states = new_h_states * (1 - done)
        new_c_states = new_c_states * (1 - done)

        hidden = hidden.view(batch_size, -1)
        new_lstm_states = (
            new_h_states.view(batch_size, -1),
            new_c_states.view(batch_size, -1),
        )

        return hidden, new_lstm_states

    def get_value(self, x, lstm_states, done):
        hidden, _ = self.get_states(x, lstm_states, done)
        hidden = self.mlp(hidden)
        return self.critic(hidden)

    def get_action_and_value(self, x, lstm_states, done, action=None):
        hidden, new_lstm_states = self.get_states(x, lstm_states, done)
        hidden = self.mlp(hidden)
        logits = self.actor(hidden)
        probs = Categorical(logits=logits)

        if action is None:
            action = probs.sample()

        return (
            action,
            probs.log_prob(action),
            probs.entropy(),
            self.critic(hidden),
            new_lstm_states,
        )

    def layer_init(self, layer, scale=np.sqrt(2), bias_const=0.0):
        torch.nn.init.orthogonal_(layer.weight, scale)
        torch.nn.init.constant_(layer.bias, bias_const)
        return layer

    def lstm_init(self):
        for name, param in self.lstm.named_parameters():
            if "bias" in name:
                nn.init.constant_(param, 0)
            elif "weight" in name:
                nn.init.orthogonal_(param, 1.0)

In [ ]:
args = Config()
args.batch_size = int(args.num_workers * args.rollout_length)
args.minibatch_size = int(args.batch_size // args.num_minibatches)
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = args.torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# env setup
envs = [make_env for _ in range(args.num_workers)]
envs = Serial(
    envs,
    [() for _ in range(args.num_workers)],
    [{} for _ in range(args.num_workers)],
    args.num_workers,
)
print(envs.single_action_space)
print(envs.single_observation_space)
envs.is_vector_env = True

agent = Agent(envs).to(device)
optimizer = optim.Adam(agent.parameters(), lr=args.adam_lr, eps=args.adam_eps)

Discrete(5)
Box(0, 255, (3, 5, 5), uint8)


In [ ]:
obs = torch.zeros(
    (args.rollout_length, args.num_workers * 2)
    + envs.single_observation_space.shape
).to(device)
actions = torch.zeros(
    (args.rollout_length, args.num_workers * 2) + envs.single_action_space.shape
).to(device)
logprobs = torch.zeros((args.rollout_length, args.num_workers * 2)).to(device)
rewards = torch.zeros((args.rollout_length, args.num_workers * 2)).to(device)
dones = torch.zeros((args.rollout_length, args.num_workers * 2)).to(device)
values = torch.zeros((args.rollout_length, args.num_workers * 2)).to(device)

lstm_state = (
    torch.zeros(agent.lstm.num_layers, args.num_workers, agent.lstm.hidden_size).to(device),
    torch.zeros(agent.lstm.num_layers, args.num_workers, agent.lstm.hidden_size).to(device),
)
lstm_state_opponent = (
    torch.zeros(agent.lstm.num_layers, args.num_workers, agent.lstm.hidden_size).to(device),
    torch.zeros(agent.lstm.num_layers, args.num_workers, agent.lstm.hidden_size).to(device),
)

# TRY NOT TO MODIFY: start the game
global_step = 0
start_time = time.time()
next_obs, info = envs.reset()
next_obs = torch.Tensor(next_obs).to(device)
next_done = torch.zeros(args.num_workers * 2).to(device)
num_updates = int(args.total_timesteps // args.batch_size)

cumulative_rewards = np.zeros(args.num_workers * 2)

In [ ]:
list(map(lambda x: x.shape, (next_obs, lstm_state[0], next_done)))

[torch.Size([16, 3, 5, 5]), torch.Size([1, 8, 256]), torch.Size([16])]

In [ ]:
def split_batch(joint_obs: torch.Tensor) -> Tuple[torch.Tensor]:
    """Splits a batch of joint data in agent and opponent data."""
    agent_data = joint_obs[[i for i in np.arange(0, args.num_workers * 2, 2)]]
    opponent_data = joint_obs[[i for i in np.arange(1, args.num_workers * 2, 2)]]
    return agent_data, opponent_data

In [ ]:
agent_obs, opponent_obs = split_batch(next_obs)
agent_done, opponent_done = split_batch(next_done)

agent.get_action_and_value(agent_obs, lstm_state, agent_done)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
assert

In [ ]:
with tqdm(total=num_updates) as pbar:
    for update in range(1, num_updates + 1):
        # Annealing the rate if instructed to do so.
        if args.anneal_lr:
            frac = 1.0 - (update - 1.0) / num_updates
            lrnow = frac * args.adam_lr
            optimizer.param_groups[0]["lr"] = lrnow

        for step in range(0, args.rollout_length):
            global_step += 1 * args.num_workers * 2
            obs[step] = next_obs
            dones[step] = next_done

            # ALGO LOGIC: action logic
            with torch.no_grad():
                action, logprob, _, value, lstm_state = agent.get_action_and_value(
                    next_obs, lstm_state, next_done
                )
                values[step] = value.flatten()
            actions[step] = action
            logprobs[step] = logprob

            # TRY NOT TO MODIFY: execute the game and log data.
            next_obs, reward, done, trunc, info = envs.step(action.cpu().numpy())
            rewards[step] = torch.tensor(reward).to(device).view(-1)
            next_obs, next_done = torch.Tensor(next_obs).to(device), torch.Tensor(
                done
            ).to(device)

            cumulative_rewards = cumulative_rewards + reward

        with torch.no_grad():
            next_value = agent.get_value(next_obs, lstm_state, next_done).reshape(1, -1)
            advantages = torch.zeros_like(rewards).to(device)
            lastgaelam = 0
            for t in reversed(range(args.rollout_length)):
                if t == args.rollout_length - 1:
                    nextnonterminal = 1.0 - next_done
                    nextvalues = next_value
                else:
                    nextnonterminal = 1.0 - dones[t + 1]
                    nextvalues = values[t + 1]
                delta = (
                    rewards[t] + args.gamma * nextvalues * nextnonterminal - values[t]
                )
                advantages[t] = lastgaelam = (
                    delta + args.gamma * args.gae_lambda * nextnonterminal * lastgaelam
                )
            returns = advantages + values

        # flatten the batch
        b_obs = obs.reshape((-1,) + envs.single_observation_space.shape)
        b_dones = dones.reshape((-1,))
        b_logprobs = logprobs.reshape(-1)
        b_actions = actions.reshape((-1,) + envs.single_action_space.shape)
        b_advantages = advantages.reshape(-1)
        b_returns = returns.reshape(-1)
        b_values = values.reshape(-1)

        # Optimizing the policy and value network
        b_inds = np.arange(args.batch_size)
        clipfracs = []
        for epoch in range(args.update_epochs):
            np.random.shuffle(b_inds)
            for start in range(0, args.batch_size, args.minibatch_size):
                end = start + args.minibatch_size
                mb_inds = b_inds[start:end]

                _, newlogprob, entropy, newvalue, lstm_state = (
                    agent.get_action_and_value(
                        b_obs[mb_inds],
                        lstm_state,
                        b_dones,
                        b_actions.long()[mb_inds],
                    )
                )
                logratio = newlogprob - b_logprobs[mb_inds]
                ratio = logratio.exp()

                with torch.no_grad():
                    # calculate approx_kl http://joschu.net/blog/kl-approx.html
                    old_approx_kl = (-logratio).mean()
                    approx_kl = ((ratio - 1) - logratio).mean()
                    clipfracs += [
                        ((ratio - 1.0).abs() > args.clip_coef).float().mean().item()
                    ]

                mb_advantages = b_advantages[mb_inds]
                if args.norm_adv:
                    mb_advantages = (mb_advantages - mb_advantages.mean()) / (
                        mb_advantages.std() + 1e-8
                    )

                # Policy loss
                pg_loss1 = -mb_advantages * ratio
                pg_loss2 = -mb_advantages * torch.clamp(
                    ratio, 1 - args.clip_coef, 1 + args.clip_coef
                )
                pg_loss = torch.max(pg_loss1, pg_loss2).mean()

                # Value loss
                newvalue = newvalue.view(-1)
                if args.clip_vloss:
                    v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                    v_clipped = b_values[mb_inds] + torch.clamp(
                        newvalue - b_values[mb_inds],
                        -args.clip_coef,
                        args.clip_coef,
                    )
                    v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                    v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                    v_loss = 0.5 * v_loss_max.mean()
                else:
                    v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()

                entropy_loss = entropy.mean()
                loss = pg_loss - args.ent_coef * entropy_loss + v_loss * args.vf_coef

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(agent.parameters(), args.max_grad_norm)
                optimizer.step()

            if args.target_kl is not None:
                if approx_kl > args.target_kl:
                    break

  0%|          | 0/48 [00:00<?, ?it/s]


RuntimeError: Expected hidden[0] size (1, 16, 256), got [1, 16, 4096]